<a href="https://colab.research.google.com/github/wjdgoruds2/Machine_Learning/blob/main/State_action_value_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np#벡터와 행렬 연산
import matplotlib.pyplot as plt#그래프 시각화 위함
import time#시간 다루기

class Environment:
  cliff=-3;road=-1;sink=-2;goal=2#절벽,길,목적지,보상 설정
  goal_position=[2,3]#목표 위치 설정
  reward_list=[[road,road,road,road],[road,road,sink,road],[road,road,road,goal]]#보상 리스트 숫자 설정
  reward_list1=[['road','road','road','road'],['road','road','sink','road'],['road','road','road','goal']]#보상 리스트 문자 설정

  def __init__(self):#보상값 생성자
    self.reward=np.asarray(self.reward_list)#보상 리스트를 배열로 설정

  def move(self,agent,action):#선택된 에이전트의 행동 결과 반환 (미로밖일 경우 이전 좌표로 다시 복귀)
    done=False
    new_pos=agent.pos+agent.action[action]#현재 위치+행동값->새로운 위치값

    if self.reward_list1[agent.pos[0]][agent.pos[1]]=='goal':#현재좌표가 목적지 인지확인
      reward=self.goal#목적지 보상값
      observation=agent.set_pos(agent.pos)#에이전트 위치 저장
      done=True
    elif new_pos[0] < 0 or new_pos[0]>=self.reward.shape[0] or new_pos[1]<0 or new_pos[1] >= self.reward.shape[1]:#이동 후 좌표가 미로 밖인 확인
      reward=self.cliff#절벽 보상값
      observation=agent.set_pos(agent.pos)#에이전트 위치 저장
      done=True
    else:#이동 후 좌표가 길이라면
      observation=agent.set_pos(new_pos)#에이전트 위치 저장
      reward=self.reward[observation[0],observation[1]]
    return observation,reward,done

class Agent:
  action=np.array([[-1,0],[0,1],[1,0],[0,-1]])#행동에 따른 에이전트의 좌표 이동(위, 오른쪽, 아래, 왼쪽)
  select_action_pr=np.array([0.25,0.25,0.25,0.25])# 각 행동별 선택확률

  def __init__(self,initial_position):#에이전트 초기 위치 저장
    self.pos=initial_position

  def set_pos(self,position):#에이전트 위치 저장
    self.pos=position
    return self.pos
  
  def get_pos(self):#에이전트 위치 불러오기
    return self.pos

def action_value_function(env,agent,act,G,max_step,now_step):
  gamma=0.9#감가율 설정

  if env.reward_list1[agent.pos[0]][agent.pos[1]] == 'goal':#현재 위치가 목적지인지 확인
    return env.goal
  if max_step==now_step:#마지막 상태는 보상만 계산
    observation,reward,done=env.move(agent,act)
    G+=agent.select_action_pr[act]*reward
    return G
  else:#현재 상태의 보상을 계산한 후 다음 행동과 함께 다음 step으로 이동
    pos1=agent.get_pos()#현재 위치 저장
    observation,reward,done=env.move(agent,act)
    G+=agent.select_action_pr[act]*reward
    
    if done==True:#이동 후 위치 확인 : 미로밖, 벽, 구멍인 경우 이동전 좌표로 다시 이동
      if observation[0]<0 or observation[0]>=env.reward.shape[0] or observation[1]<0 or observation[1]>=env.reward.shape[1]:
        agent.set_pos(pos1)

    pos1=agent.get_pos()#현재 위치를 다시 저장

    for i in range(len(agent.action)):#현재 위치에서 가능한 모든 행동을 선택한 후 이동
      agent.set_pos(pos1)
      next_v=action_value_function(env,agent,i,0,max_step,now_step+1)
      G+=agent.select_action_pr[i]*gamma*next_v
    return G

def show_q_table(q_table,env):# Q table 그리기
  for i in range(env.reward.shape[0]):
    print('+-----------------------'*env.reward.shape[1],end='');print('+')
    for k in range(3):
      print('|',end='')
      for j in range(env.reward.shape[1]):
        if k==0:
          print('{0:10.2f}   |'.format(q_table[i,j,0]),end='')
        if k==1:
          print('{0:6.2f}{1:6.2f} |'.format(q_table[i,j,3],q_table[i,j,1]),end='')
        if k==2:
          print('{0:10.2f}   |'.format(q_table[i,j,2]),end='')
      print()
  print('+-----------------------'*env.reward.shape[1],end='');print('+')

def show_q_table_arrow(q_table,env):# 정책 policy 화살표로 그리기
  for i in range(env.reward.shape[0]):
    print('+-----------------------'*env.reward.shape[1],end='');print('+')
    for k in range(3):
      print('|',end='')
      for j in range(env.reward.shape[1]):
        if k==0:
          if np.max(q[i,j,:]) == q[i,j,0]:
            print('       ↑      |',end='')
          else:
            print('               |',end='')
        if k==1:
            if np.max(q[i,j,:]) == q[i,j,1] and np.max(q[i,j,:])==q[i,j,3]:
              print('     ←  →    |',end='')
            elif np.max(q[i,j,:]) == q[i,j,1]:
              print('        →     |',end='')
            elif np.max(q[i,j,:]) == q[i,j,3]:
              print('     ←        |',end='')
            else:
              print('               |',end='') 
        if k==2:
            if np.max(q[i,j,:]) == q[i,j,2]:
              print('       ↓      |',end='')
            else:
              print('               |',end='') 
      print()
  print('+-----------------------'*env.reward.shape[1],end='');print('+')

env=Environment()#환경 초기화
initial_position=np.array([0,0])#초기값 설정
agent=Agent(initial_position)#에이전트 초기화
np.random.seed(0)
max_step_number=8#현재부터 max_step 까지 계산

for max_step in range(max_step_number):#모든 상태에 대해
  print('max_step={}'.format(max_step))
  q_table=np.zeros((env.reward.shape[0],env.reward.shape[1],len(agent.action)))# 미로 상의 모든 상태에서 가능한 행동의 가치를 저장할 테이블을 정의
  for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
      for action in range(len(agent.action)):#모든 행동에 대해
        agent.set_pos([i,j])#에이전트의 위치를 초기화
        q_table[i,j,action]=action_value_function(env,agent,action,0,max_step,0)#현재 위치에서 행동 가치를 계산
  q=np.round(q_table,2)
  print('\nQ-table')
  show_q_table(q,env)#Q테이블 출력
  print('\n정책')
  show_q_table_arrow(q,env)#화살표 방향 출력


max_step=0

Q-table
+-----------------------+-----------------------+-----------------------+-----------------------+
|     -0.75   |     -0.75   |     -0.75   |     -0.75   |
| -0.75 -0.25 | -0.25 -0.25 | -0.25 -0.25 | -0.25 -0.75 |
|     -0.25   |     -0.25   |     -0.50   |     -0.25   |
+-----------------------+-----------------------+-----------------------+-----------------------+
|     -0.25   |     -0.25   |     -0.25   |     -0.25   |
| -0.75 -0.25 | -0.25 -0.50 | -0.25 -0.25 | -0.50 -0.75 |
|     -0.25   |     -0.25   |     -0.25   |      0.50   |
+-----------------------+-----------------------+-----------------------+-----------------------+
|     -0.25   |     -0.25   |     -0.50   |      2.00   |
| -0.75 -0.25 | -0.25 -0.25 | -0.25  0.50 |  2.00  2.00 |
|     -0.75   |     -0.75   |     -0.75   |      2.00   |
+-----------------------+-----------------------+-----------------------+-----------------------+

정책
+-----------------------+-----------------------+-------------